## Install Required Libraries
Install any necessary libraries that are not pre-installed in the environment.

In [1]:
!pip install transformers datasets

## Load the Model and Tokenizer
Many LLM models are available in the Hugging Face model hub. We can load the model needed and its corresponding tokenizer using the AutoModelForCausalLM and AutoTokenizer classes.

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

## this is too big
# base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Load a pre-trained model
model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model.to("cuda")

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

## Prepare the Input
To generate text autoregressively, wou need to provide an initial prompt or input sequence.

In [3]:
# Define a prompt
def apply_prompt(example):
    prompt = f"""
    ### Question: {example['question']}
    ### Context: {example['context']}
    ### Answer: {example['answer']}
    """
    return prompt.strip()  # Remove leading/trailing whitespace

In [4]:
question = "What are the financial consequences of exchange rate fluctuations?"

# Tokenize the input
inputs = tokenizer(apply_prompt({"question":question, "context":"", "answer":""}), return_tensors="pt").to("cuda")
# return_tensors="pt" for Returns PyTorch tensors, use `return_tensors="tf"` for TensorFlow

## Generate Text
Use the generate method to produce text. You can control the behavior of text generation using various parameters:

In [5]:
# Generate text
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,  # Maximum length of the generated answer
    use_cache=True,
    # num_beams=5,          # Beam search for better quality
    # temperature=0.7,      # Controls randomness (lower = more deterministic)
    # early_stopping=True,  # Stop generation when the model outputs the end-of-sequence token
)

# Decode the generated tokens to text
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer = answer.split("### Answer:")[-1].strip()
print(answer)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


The financial consequences of exchange rate fluctuations can be both positive and negative. On the positive side, exchange rate fluctuations can lead to higher profits in international trade, as well as increased revenue for companies that export goods and services. Additionally, exchange rate fluctuations can improve the performance of international bonds, as the value of the bonds increases with a stronger currency. On the negative side, exchange rate fluctuations can result in losses for companies that rely on foreign currency operations, such as a country's export companies. Moreover, exchange rate fluctuations can lead to a decline in the value of foreign assets, such as a country's government bonds, as the currency is weaker. Additionally, exchange rate fluctuations can lead to a decrease in the value of foreign currencies, which can cause a loss for individuals holding those currencies. Finally, exchange rate fluctuations can result in a loss for individuals who hold foreign ass

## Load the Dataset
Load your dataset. You can use the datasets library from Hugging Face for easy loading and preprocessing.

In [6]:
from datasets import load_dataset
dataset = load_dataset("virattt/financial-qa-10K",split = "train")

dataset = dataset.map(lambda example: {"prompt": apply_prompt(example)})
print(dataset[0]["prompt"])

README.md:   0%|          | 0.00/419 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

### Question: What area did NVIDIA initially focus on before expanding to other computationally intensive fields?
    ### Context: Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.
    ### Answer: NVIDIA initially focused on PC graphics.


## Preprocessing the Dataset
Tokenize the dataset using the tokenizer:

In [7]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

## Train Model

In [8]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine-tune the model
trainer.train()

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/parallel_apply.py", line 96, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/qwen2/modeling_qwen2.py", line 1165, in forward
    outputs = self.model(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "

## Model inference after fine-tuning

In [ ]:
question = "What are the financial consequences of exchange rate fluctuations?"

# Tokenize the input
inputs = tokenizer(apply_prompt({"question":question, "context":"", "answer":""}), return_tensors="pt").to("cuda")

# Generate text
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# Decode the generated tokens to text
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer = answer.split("### Answer:")[-1].strip()
print(answer)

## Save model locally

In [ ]:
new_model_name = "DeepSeek-R1-Financial"
model.save_pretrained(new_model_name) # Local saving
tokenizer.save_pretrained(new_model_name)
model.save_pretrained_merged(new_model_name, tokenizer, save_method = "merged_16bit")

## Push model to HuggingFace hub

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")


from huggingface_hub import login
login(hf_token)

In [ ]:
hf_model_name = f"wangbn/{new_model_name}"

model.push_to_hub(hf_model_name) # Online saving
tokenizer.push_to_hub(hf_model_name) # Online saving
model.push_to_hub_merged(hf_model_name, tokenizer, save_method = "merged_16bit")